#  Demo of Ridge Regression using Stochasitc Gradient Descent optimization

You will need to play with the `learning_rate` and `l2_lambda` (regularization parameter) hyperparameters in order to get a fast and stable optimizaiton.

* Small batches allow us to get quick estimates of the true gradient (since we only have to process one batch worth of data, instead of the whole data set).

* Randomization allows us to avoid pathalogical orderings of the data that might cause us to go off in the wrong direction.

* If we are solving a non-convex problem (e.g. not this case), then randomization can also help to avoid and escape local minima

In [8]:
from IPython.display import Latex


Our minimization problem is
$$
\min_\theta \mathcal{L}
$$
where our loss function is
$$
\mathcal{L} = \frac{1}{m} \|X\theta - y\|^2 + \lambda\|\theta\|^2
$$

The gradient is 
$$
\nabla_{\theta} \mathcal{L} = \frac{2}{m} X^T (X\theta - y) + 2\lambda\theta
$$

If we designate our error term as
$$
\hat{\varepsilon} = X\theta - y
$$
and we drop the factor of 2 (since that can just be folded into the learning rate) then we have
$$
\nabla_{\theta} \mathcal{L} = \frac{1}{m} X^T \hat{\varepsilon} + \lambda\theta
$$

This formulation corresponds directly to the implementations below



## On CPU using Pandas

Pandas + Numpy is often easiest for smaller problems

In [9]:
import numpy as np
import pandas as pd
import timeit

# Generate synthetic dataset
n_samples = 800
n_features = 50_000

# Features
X = pd.DataFrame(np.random.randn(n_samples, n_features))
X = (X - X.mean()) / X.std()

# True weights
true_weights = np.random.randn(n_features)

# Target variable with noise
y = pd.Series(X.values @ true_weights + 0.1 * np.random.randn(n_samples))


In [10]:


# Initialize weights
weights = np.zeros(n_features)
learning_rate = 1e-4
n_epochs = 50
batch_size = 32

# Timing start
start_time = timeit.default_timer()

# Training loop
for epoch in range(n_epochs):
  # Shuffle the data
  indices = np.random.permutation(n_samples)
  X_shuffled = X.values[indices]
  y_shuffled = y.values[indices]

  for i in range(0, n_samples, batch_size):
    X_batch = X_shuffled[i:i+batch_size]
    y_batch = y_shuffled[i:i+batch_size]

    # Predict
    y_pred = X_batch @ weights

    # Compute error
    error = y_pred - y_batch

    l2_lambda = 1e-3

    # Compute gradient
    gradient = X_batch.T @ error / batch_size + l2_lambda * weights

    # Update weights
    weights -= learning_rate * gradient

  # Compute loss
  y_full_pred = X.values @ weights
  loss = np.mean((y_full_pred - y.values) ** 2)
  print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

# Timing end
end_time = timeit.default_timer()

print(f"Training completed in {end_time - start_time:.2f} seconds")


Epoch 1, Loss: 35906.7735
Epoch 2, Loss: 25504.9072
Epoch 3, Loss: 18146.6122
Epoch 4, Loss: 12933.8487
Epoch 5, Loss: 9235.0068
Epoch 6, Loss: 6604.7049
Epoch 7, Loss: 4731.2204
Epoch 8, Loss: 3395.0024
Epoch 9, Loss: 2440.1197
Epoch 10, Loss: 1756.6248
Epoch 11, Loss: 1266.6852
Epoch 12, Loss: 914.8037
Epoch 13, Loss: 661.6896
Epoch 14, Loss: 479.3859
Epoch 15, Loss: 347.7772
Epoch 16, Loss: 252.6872
Epoch 17, Loss: 183.8635
Epoch 18, Loss: 133.9741
Epoch 19, Loss: 97.7534
Epoch 20, Loss: 71.4237
Epoch 21, Loss: 52.2548
Epoch 22, Loss: 38.2787
Epoch 23, Loss: 28.0762
Epoch 24, Loss: 20.6184
Epoch 25, Loss: 15.1595
Epoch 26, Loss: 11.1595
Epoch 27, Loss: 8.2246
Epoch 28, Loss: 6.0678
Epoch 29, Loss: 4.4817
Epoch 30, Loss: 3.3136
Epoch 31, Loss: 2.4525
Epoch 32, Loss: 1.8171
Epoch 33, Loss: 1.3476
Epoch 34, Loss: 1.0003
Epoch 35, Loss: 0.7433
Epoch 36, Loss: 0.5529
Epoch 37, Loss: 0.4116
Epoch 38, Loss: 0.3067
Epoch 39, Loss: 0.2288
Epoch 40, Loss: 0.1708
Epoch 41, Loss: 0.1277
Epoch 4

In [11]:

X_mean = X.mean(axis=0).values
feature_contributions = (X - X_mean) * weights  # shape (n_samples, n_features)

print("Top 10 features by SHAP / feature importance")
print(
  feature_contributions
  .abs()
  .mean(axis=0)
  .sort_values(ascending=False).head(10)
)

Top 10 features by SHAP / feature importance
47415    0.444863
29437    0.408179
23284    0.405233
45375    0.380249
9019     0.378597
14792    0.376864
32212    0.370889
41712    0.370864
24048    0.365077
37256    0.364913
dtype: float64


## Using PyTorch

Pytorch makes it really easy to parallize onto GPUs or using the parallel processing capabilities of Apple processors (MPS)

In [12]:
import torch
import timeit

# Select device (GPU if available, else CPU)
device = (
  torch.device("cuda")
  if torch.cuda.is_available()
  else torch.device("mps")
  if torch.backends.mps.is_available()
  else torch.device("cpu")
)

print(f"Using device: {device}")

X_tensor = torch.tensor(X.values.astype(np.float32), device=device)
y_tensor = torch.tensor(y.values.astype(np.float32), device=device)

# Initialize model parameters
weights = torch.zeros(n_features, device=device, requires_grad=True)

learning_rate = 1e-4
n_epochs = 50
batch_size = 32
l2_lambda = 1e-3

# Timing start
start_time = timeit.default_timer()

# Training loop
for epoch in range(n_epochs):
  perm = torch.randperm(n_samples, device=device)
  X_shuffled = X_tensor[perm]
  y_shuffled = y_tensor[perm]

  for i in range(0, n_samples, batch_size):
    X_batch = X_shuffled[i:i+batch_size]
    y_batch = y_shuffled[i:i+batch_size]

    y_pred = X_batch @ weights
    error = y_pred - y_batch

    loss = (error ** 2).mean() + l2_lambda * (weights ** 2).sum()

    # Backpropagation - this calculates the gradients for all the tensors with requires_grad=True
    loss.backward()

    # SGD step - this is where we update the weights
    with torch.no_grad():
      weights -= learning_rate * weights.grad
      weights.grad.zero_()

  # Compute full loss
  with torch.no_grad():
    y_full_pred = X_tensor @ weights
    full_loss = ((y_full_pred - y_tensor) ** 2).mean().item()
    print(f"Epoch {epoch+1}, Loss: {full_loss:.4f}")

# Timing end
end_time = timeit.default_timer()

print(f"Training completed in {end_time - start_time:.2f} seconds")


Using device: mps
Epoch 1, Loss: 23782.8242
Epoch 2, Loss: 11265.2275
Epoch 3, Loss: 5383.1626
Epoch 4, Loss: 2593.4277
Epoch 5, Loss: 1260.0693
Epoch 6, Loss: 617.2495
Epoch 7, Loss: 304.6308
Epoch 8, Loss: 151.4463
Epoch 9, Loss: 75.8233
Epoch 10, Loss: 38.2100
Epoch 11, Loss: 19.3741
Epoch 12, Loss: 9.8788
Epoch 13, Loss: 5.0659
Epoch 14, Loss: 2.6123
Epoch 15, Loss: 1.3531
Epoch 16, Loss: 0.7045
Epoch 17, Loss: 0.3684
Epoch 18, Loss: 0.1935
Epoch 19, Loss: 0.1022
Epoch 20, Loss: 0.0542
Epoch 21, Loss: 0.0289
Epoch 22, Loss: 0.0155
Epoch 23, Loss: 0.0084
Epoch 24, Loss: 0.0045
Epoch 25, Loss: 0.0025
Epoch 26, Loss: 0.0014
Epoch 27, Loss: 0.0008
Epoch 28, Loss: 0.0005
Epoch 29, Loss: 0.0003
Epoch 30, Loss: 0.0002
Epoch 31, Loss: 0.0001
Epoch 32, Loss: 0.0001
Epoch 33, Loss: 0.0001
Epoch 34, Loss: 0.0000
Epoch 35, Loss: 0.0000
Epoch 36, Loss: 0.0000
Epoch 37, Loss: 0.0000
Epoch 38, Loss: 0.0000
Epoch 39, Loss: 0.0000
Epoch 40, Loss: 0.0000
Epoch 41, Loss: 0.0000
Epoch 42, Loss: 0.0000

ALthough the total time is longer, it converges much faster (presumably because it uses better gradients), so if we were stopping on an epsilon, it would probably be faster.

In [13]:
X_tensor_mean = X_tensor.mean(axis=0)
feature_contributions_torch = (X_tensor - X_tensor_mean) * weights  # shape (n_samples, n_features)

print("Top 10 features by SHAP / feature importance")
print(
  pd.Series(
    feature_contributions_torch
    .abs()
    .mean(axis=0)
    .cpu()
    .detach()
    .numpy()
  )
  .sort_values(ascending=False)
  .head(10)
)

Top 10 features by SHAP / feature importance
47415    0.445028
29437    0.408296
23284    0.405386
45375    0.380387
9019     0.378700
14792    0.377018
32212    0.371025
41712    0.370977
24048    0.365115
37256    0.365078
dtype: float32


## Using pseudo inverse via SVD with L2 regularization



In [14]:
import numpy as np


# Timing start
start_time = timeit.default_timer()

# Compute SVD
U, S, Vt = np.linalg.svd(X, full_matrices=False)

# Compute pseudo-inverse of the singular values
l2_lambda = 1e-3  # Same as other methods
S_inv = np.diag(S / (S**2 + l2_lambda))  # Regularized version

# Compute pseudo-inverse of X
X_pinv = Vt.T @ S_inv @ U.T

# Solve for weights
w_svd = X_pinv @ y

# Timing end
end_time = timeit.default_timer()

# Check training loss
y_pred = X @ w_svd
loss = np.mean((y_pred - y) ** 2)
print(f"Training Loss (SVD pseudo-inverse): {loss:.4f}")

print(f"Training completed in {end_time - start_time:.2f} seconds")


Training Loss (SVD pseudo-inverse): 0.0000
Training completed in 4.18 seconds


In [15]:
feature_contributions_svd = (X - X_mean) * w_svd  # shape (n_samples, n_features)

print("Top 10 features by SHAP / feature importance")
print(
  feature_contributions_svd
  .abs()
  .mean(axis=0)
  .sort_values(ascending=False).head(10)
)

Top 10 features by SHAP / feature importance
47415    0.445034
29437    0.408301
23284    0.405391
45375    0.380392
9019     0.378705
14792    0.377023
32212    0.371031
41712    0.370982
24048    0.365119
37256    0.365084
dtype: float64
